In [1]:
import os
import pickle
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.classifiers import Classifier
import tempfile
import traceback
import weka.core.serialization as serialization
from weka.classifiers import Classifier
from weka.core.classes import Random
import weka.core.jvm as jvm
from weka.core.classes import Random
from weka.core.converters import Loader
from weka.classifiers import Classifier, Evaluation, PredictionOutput, Prediction
import arff
import pandas as pd
import numpy as np

In [2]:
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['D:\\Anaconda\\envs\\pww3\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'D:\\Anaconda\\envs\\pww3\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'D:\\Anaconda\\envs\\pww3\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'd:\\Anaconda\\envs\\pww3\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'd:\\Anaconda\\envs\\pww3\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [3]:
data_dir = r'..\data\adFontesOUT.arff'
loader = Loader(classname="weka.core.converters.ArffLoader")
data = loader.load_file(data_dir)
data.class_is_last()
train, test = data.train_test_split(66.0, Random(1))

In [4]:
classifier = Classifier("weka.classifiers.lazy.IBk", options = ["-E", "-K", "11", "-I"])

In [5]:
classifier.build_classifier(train)

In [6]:
print(classifier)

IB1 instance-based classifier
using 11 inverse-distance-weighted nearest neighbour(s) for classification



In [15]:
outfile = 'prediction.csv'
output = PredictionOutput(classname="weka.classifiers.evaluation.output.prediction.CSV", options=["-distribution", "-suppress", "-file", outfile])
output.header = test[0]
output.print_classification(classifier, test[0], 0)
print(output.buffer_content())

In [8]:
#data = ['', '', '', "The Quick Brown Fox Jumped Over the Lazy Dog"]
data = ['0', '', '', '',"In all sincerity, I like Americans a lot; I’ve met many lovely people in the United States, and I empathize with the shame many Americans (and not only “New York intellectuals”) feel at having such an appalling clown for a leader. However, I have to ask—and I know what I’m requesting isn’t easy for you—that you consider things for a moment from a non-American point of view. I don’t mean “from a French point of view,” which would be asking too much; let’s say, “from the point of view of the rest of the world.” On the numerous occasions when I’ve been questioned about Donald Trump’s election, I’ve replied that I don’t give a shit. France isn’t Wyoming or Arkansas. France is an independent country, more or less, and will become totally independent once again when the European Union is dissolved (the sooner, the better). The United States of America is no longer the world’s leading power. It was for a long time, for almost the entire course of the twentieth century. It isn’t anymore. It remains a major power, one among several. This isn’t necessarily bad news for Americans. It’s very good news for the rest of the world. My response is a bit of an exaggeration. One has an ongoing obligation to take at least a modicum of interest in American political life. The United States is still the world’s leading military power and unfortunately has yet to break its habit of mounting interventions beyond its borders. I’m not a historian, and I don’t know much about ancient history—for example, I couldn’t say whether Kennedy or Johnson was more to blame for the dismal Vietnam affair—but I have the impression that it’s been a good long time since the United States last won a war, and that for at least fifty years its foreign military interventions, whether acknowledged or clandestine, have been nothing but a succession of disgraces culminating in failures. Let’s go back all the way to the United States’s last morally unquestionable and militarily victorious intervention, namely its participation in World War II: What would have happened had the United States not entered the war (an unpleasant alternate history)? Without a doubt, the destiny of Asia would have been greatly altered. The destiny of Europe, too, but probably somewhat less. In any case, Hitler would have lost just the same. What’s most probable is that Stalin’s armies would have reached Cherbourg. Some European countries that were spared the ordeal of communism would have suffered it."]
#data = ['Fox News', '', 'Donald Trump Is a Good President', "In all sincerity, I like Americans a lot; I’ve met many lovely people in the United States, and I empathize with the shame many Americans (and not only “New York intellectuals”) feel at having such an appalling clown for a leader. However, I have to ask—and I know what I’m requesting isn’t easy for you—that you consider things for a moment from a non-American point of view. I don’t mean “from a French point of view,” which would be asking too much; let’s say, “from the point of view of the rest of the world.” On the numerous occasions when I’ve been questioned about Donald Trump’s election, I’ve replied that I don’t give a shit. France isn’t Wyoming or Arkansas. France is an independent country, more or less, and will become totally independent once again when the European Union is dissolved (the sooner, the better). The United States of America is no longer the world’s leading power. It was for a long time, for almost the entire course of the twentieth century. It isn’t anymore. It remains a major power, one among several. This isn’t necessarily bad news for Americans. It’s very good news for the rest of the world. My response is a bit of an exaggeration. One has an ongoing obligation to take at least a modicum of interest in American political life. The United States is still the world’s leading military power and unfortunately has yet to break its habit of mounting interventions beyond its borders. I’m not a historian, and I don’t know much about ancient history—for example, I couldn’t say whether Kennedy or Johnson was more to blame for the dismal Vietnam affair—but I have the impression that it’s been a good long time since the United States last won a war, and that for at least fifty years its foreign military interventions, whether acknowledged or clandestine, have been nothing but a succession of disgraces culminating in failures. Let’s go back all the way to the United States’s last morally unquestionable and militarily victorious intervention, namely its participation in World War II: What would have happened had the United States not entered the war (an unpleasant alternate history)? Without a doubt, the destiny of Asia would have been greatly altered. The destiny of Europe, too, but probably somewhat less. In any case, Hitler would have lost just the same. What’s most probable is that Stalin’s armies would have reached Cherbourg. Some European countries that were spared the ordeal of communism would have suffered it."]
df = pd.DataFrame(columns=['index', 'source', 'label', 'headline', 'text'])
df.loc[0] = data
print(df.head)
arff.dump('data.arff',
        df.values,
        names=df.columns
)

<bound method NDFrame.head of   index source label headline  \
0     0                         

                                                text  
0  In all sincerity, I like Americans a lot; I’ve...  >


In [9]:
path = 'data.arff'
loader = Loader(classname="weka.core.converters.ArffLoader")
data = loader.load_file(path)

print("# - actual - predicted - error - distribution")
for index, inst in enumerate(test):
    pred = classifier.classify_instance(inst)
    dist = classifier.distribution_for_instance(inst)
    print(
        "%d - %s - %s - %s  - %s" %
        (index+1,
         inst.get_string_value(inst.class_index),
         inst.class_attribute.value(int(pred)),
         "yes" if pred != inst.get_value(inst.class_index) else "no",
         str(dist.tolist())))

# - actual - predicted - error - distribution


AttributeError: 'str' object has no attribute 'jobject'

In [10]:
output = PredictionOutput(classname="weka.classifiers.evaluation.output.prediction.CSV", options=["-distribution"])
evl = Evaluation(train)
evl.test_model(classifier, test[:10], output=output)
print(evl.summary())
print(output.buffer_content())


Correctly Classified Instances           5               50      %
Incorrectly Classified Instances         5               50      %
Kappa statistic                          0.4681
Mean absolute error                      0.0003
Root mean squared error                  0.0122
Relative absolute error                 74.6196 %
Root relative squared error             83.3642 %
Total Number of Instances               10     

1,2294:'WASHINGTON  The Republicanled House of Representatives approved a farreaching bill to ban a womans ability to seek an abortion after  weeks on a mostly partyline  vote Tuesday It stands no chance of becoming law under the Obama administration The White House issued a veto threat Monday calling the bill an assault on a womans right to choose The Democratic Senate also has not scheduled a vote on the legislation However House Speaker John Boehner ROhio justified the vote as a response to the recent murder conviction of Philadelphia physician Kermit Gosnell for

In [11]:
#jvm.stop()